# Hello world

In [3]:
import os
import openai

def GPT_Completion(texts):## Call the API key under your account (in a secure way)
    openai.api_key = "***REMOVED***"

    response = openai.Completion.create(engine="text-davinci-002",
                                        prompt =  texts,
                                        temperature = 0.6,
                                        top_p = 1,
                                        max_tokens = 64,
                                        frequency_penalty = 0,
                                        presence_penalty = 0)
    return print(response.choices[0].text)

recipe = 'Provide a cooking recipe based on the following ingredients: \\n \nApple \
          \n \nFlour \
          \n \nChicken \
          \n \nSalt'

GPT_Completion(recipe)



Apple Chicken

Ingredients:

1 apple, cored and sliced
1 chicken breast, cut into small pieces
1/4 cup flour
1/4 teaspoon salt

Instructions:

1. Preheat oven to 375 degrees F.
2. In a medium bowl, combine


# Generate training dataset

## Generate problems (non filtra i duplicati)

In [ ]:
import os
import filecmp

## 8000 data points should improve performance between 2x and 4x

%run generator/pddl_confgen.py 4 8000 -r 24 -co
!mv -v generator/problems/conditional/* dataset/problems > /dev/null

## clean duplicates

problems = sorted(os.listdir('./dataset/problems/'))

duplicates = 0
to_be_removed = []

for i, problem_i in enumerate(problems):
    print("Cleaning --- " + str(round((i / 8000 * 100), 1)) +"%", end = '\r')
    for problem_j in problems[i + 1:]:
        if filecmp.cmp('./dataset/problems/' + problem_i, 
                       './dataset/problems/' + problem_j, 
                       shallow=False):
            duplicates += 1
            to_be_removed.append(problem_j)
            
for problem in to_be_removed:
    try:
        os.remove('./dataset/problems/' + problem)
    except FileNotFoundError:
        continue
            
print(str(duplicates) + " duplicates deleted")

Writing conditional problem files to /home/alessio/planning/teriyaki/generator/problems/conditional...
Done.
All operations completed. Happy planning!


## Generate plans with Probe

In [ ]:
import os
import subprocess

problems = sorted(os.listdir('./dataset/problems/'))

for i, problem in enumerate(problems):
    result = subprocess.run(['./generator/probe', 
                             '-d', './dataset/domains/domain_m_25_B.pddl', 
                             '-i', './dataset/problems/' + problem,
                             '-o', './dataset/plans/m_25_B-' + str(problem.split('_',2)[2:][0])], 
                             stdout=subprocess.PIPE)
    print("Planning --- " + str(round((i / 8000 * 100), 1)) +"%", end = '\r')


## Convert Probe plans to IPC format

In [15]:
import os

plans = sorted(os.listdir('./dataset/plans/'))

for plan in plans:
    probe = open('./dataset/plans/' + plan, "r")
    val = open('./dataset/plans_ipc/' + plan.split(".")[0] + ".plan", 'w+')
    replacement = ""
    # using the for loop
    for i, line in enumerate(probe):
        line = line.lower()
        line = '%.3f' % ((i + 1)/1000) + "00: " + line
        val.write(line)

    probe.close()
    # os.remove('./dataset/plans/' + plan)
    val.close()
    
print("Plans converted to IPC format")

Plans converted to IPC format


## Run VAL and filter out invalid plans (NON FUNZIA)

In [22]:
import os
import subprocess

problems = sorted(os.listdir('./dataset/problems/'))
invalid_plans = 0

for i, problem in enumerate(problems):
    result = subprocess.run(['./generator/Validate', 
                             './dataset/domains/domain_m_25_B.pddl', 
                             './dataset/problems/' + problem,
                             './dataset/plans_ipc/m_25_B-' + problem.split('_',2)[2:][0].split(".")[0] + ".plan"], 
                             stdout=subprocess.PIPE)
    if "Plan failed to execute" in str(result):
        invalid_plans += 1
#         os.remove('./dataset/plans_ipc/m_25_B-' + problem.split('_',2)[2:][0].split(".")[0] + ".plan")
#         os.remove('./dataset/problems/' + problem)

    print("Validating --- " + str(round((i / 8000 * 100), 1)) +"%", end = '\r')

print('%i plans could not be validated and have been removed from the dataset' % invalid_plans)

7974 plans could not be validated and have been removed from the dataset


## Compile GPT3 training dataset

In [84]:
import os
import subprocess

dataset = open('./dataset/formatted/teriyaki.jsonl', 'w+')

# extracting prompts from problems and completions from IPC plans 
for i, problem in enumerate(problems):
    problem_f = open('./dataset/problems/' + problem, "r")
    plan_f = open('./dataset/plans_ipc/m_25_B-' + problem.split('_',2)[2:][0].split(".")[0] + ".plan", "r")
    lines = problem_f.readlines()
    
    # reformat (free gleft)  (free gright)
    # lines[52] = lines[52].replace(") (", ")\n    (")
    
    # lines[49:62] is the range of meaningful predicates at initial state
    dataset.write(repr('{"prompt":"\n(:init\n' + ''.join(lines[49:62]).replace("    ", "") + 
          '\n\n###\n\n", "completion":" ' + plan_f.read().replace("    ", "") + ' END"}')[1:-1] + "\n")

dataset.close()

print("Dataset ready!")
print("Before training, run in a terminal\n\t>openai tools fine_tunes.prepare_data -f './dataset/formatted/teriyaki.jsonl'")

Dataset ready!
Before training, run in a terminal
	>openai tools fine_tunes.prepare_data -f './dataset/formatted/teriyaki.jsonl'


## Compute training cost (1 token ~= 4 chars; 0.0300$/1K Tokens)